 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#Reference" data-toc-modified-id="Reference-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Reference</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#Code" data-toc-modified-id="Code-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Code</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#generate-graph" data-toc-modified-id="generate-graph-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>generate graph</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#simple-neural-network" data-toc-modified-id="simple-neural-network-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>simple neural network</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/17F532IE/PageRank/Draft.ipynb#测试前要看一下" data-toc-modified-id="测试前要看一下-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>测试前要看一下</a></span></li></ul></li></ul></li></ul></div>

# Reference

https://medium.com/@bgoncalves/network-effects-explained-pagerank-and-preferential-attachment-61fdf93d023a

# Code

## generate graph

In [1]:
import networkx as nx

In [4]:
with open("web-Stanford.txt","r") as fil:
    for _ in range(4):
        next(fil)
    data=[tuple([int(j) for j in i.strip().split()]) for i in fil]

In [5]:
G = nx.DiGraph()
G.add_edges_from(data)

pr = nx.pagerank(G, alpha=0.9)   #作为true response?

In [6]:
for i in pr.keys():
    G[i]['score']=pr[i]

In [7]:
G[1]['score']

3.561215144991757e-07

In [8]:
n=len(G.nodes())

### Preferential Attachment Model
命名和上一个一样

In [13]:
G_temp = nx.barabasi_albert_graph(1000, 1)

# initialize an empty directed graph
G = nx.DiGraph()
G.add_edges_from(G_temp.edges())

pr = nx.pagerank(G, alpha = 0.85) 
for i in pr.keys():
    G[i]['score']=pr[i]

In [14]:
G[1]['score']

0.0007987986538502552

In [15]:
n=len(G.nodes())

## simple neural network

In [11]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf
from gcn.utils import *
from gcn.models import Model,MLP
from gcn.layers import *

import scipy.sparse as sp
import random
import numpy as np

### 测试前要看一下
+ 之前学习率填了0.05，明显训练的时候有了误差来回变大减小又变大再减小的情况，应该要调小一点，之前作业里是0.01,我一时兴起又改大了……
+ 用networkx里的pagerank算法出来的分数是在0-1之间，所以最后的误差出来即使是0.几，也不确定是不是很好，这里是100个点做validation。
+ epoch可能也要调，学习率0.05的时候300差不多收敛了，但是不知道学习率改了之后是什么情况

In [12]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'  #这行应该并没有什么用
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.05, 'Initial learning rate.')
#flags.DEFINE_float('learning_rate', 0.1, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 300, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
#flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
#flags.DEFINE_float('weight_decay', 0, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 100, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [16]:
def masked_softmax_cross_entropy(preds, labels, mask):
    """Softmax cross-entropy loss with masking."""
    # Measures the probability error in discrete classification tasks in which the classes are mutually exclusive 
    # 这里还可以试别的loss function
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=labels) 
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

#--------------- Your code here --------------#

def masked_mean_square_error(preds,labels,mask):
    """L2 loss with masking."""
    loss = tf.nn.l2_loss(preds - labels)   
    # loss = tf.nn.relu(preds - labels) Rulu loss
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)


#--------------------------------------------#

def masked_accuracy(preds, labels, mask):
    """Accuracy with masking."""
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [8]:
class GCN(Model):
    def __init__(self, placeholders, input_dim, **kwargs):
        super(GCN, self).__init__(**kwargs)

        self.inputs = placeholders['features']
        self.input_dim = input_dim
        # self.input_dim = self.inputs.get_shape().as_list()[1]  # To be supported in future Tensorflow versions
        self.output_dim = placeholders['labels'].get_shape().as_list()[1]
        self.placeholders = placeholders

        self.optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

        self.build()

    def _loss(self):
        # Weight decay loss
        for var in self.layers[0].vars.values():
            self.loss += FLAGS.weight_decay * tf.nn.l2_loss(var)

        # Cross entropy error
        self.loss += masked_mean_square_error(self.outputs, self.placeholders['labels'],
                                                 self.placeholders['labels_mask'])
#        self.loss += masked_softmax_cross_entropy(self.outputs, self.placeholders['labels'],
#                                                  self.placeholders['labels_mask'])

    def _accuracy(self):
        self.accuracy = masked_accuracy(self.outputs, self.placeholders['labels'],
                                        self.placeholders['labels_mask'])

    def _build(self):

        self.layers.append(GraphConvolution(input_dim=self.input_dim,
                                            output_dim=FLAGS.hidden1,
                                            placeholders=self.placeholders,
                                            act=tf.nn.relu,
                                            dropout=True,
                                            sparse_inputs=True,
                                            logging=self.logging))

        self.layers.append(GraphConvolution(input_dim=FLAGS.hidden1,
                                            output_dim=self.output_dim,
                                            placeholders=self.placeholders,
                                            act=lambda x: x,
                                            dropout=True,
                                            logging=self.logging))

    def predict(self):
        return self.outputs    #tf.nn.softmax(self.outputs)

In [9]:
def preprocess_adj(adj):
    """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)

# A^2
#    adj_square = np.power(adj,2).tocoo()
#    return sparse_to_tuple(adj)

In [18]:
# Randomly generate graph with 500 nodes
adj = nx.adjacency_matrix(G)

# Generate feature matrix  #there is difference between different version of networkx, 这里是用networkx 2.0. 原来1.1的用法注释掉了
#features = sp.vstack(list(j[1] for j in G.in_degree(G.nodes())),dtype = 'float32').tolil()
features = sp.vstack(list(G.in_degree(G.nodes()).values()),dtype = 'float32').tolil()   

# Feed labels to 20 nodes
label = []
for i in G.nodes():
    #label.append(G.nodes[i]['score']) 我用的1.1语法-YG
    label.append(G[i]['score'])
        
alabel = np.array(label, dtype='float64')

idx_train = range(500)
idx_val = range(500, 500 + 100) #20000改到500 for preferential attachment model

y_train = np.zeros([n, 1])   #n is number of total nodes, =281903
train_mask = np.zeros([n, ], dtype = 'bool')
for i in idx_train:
    y_train[i] = alabel[i]
    train_mask[i] = True

y_val = np.zeros([n, 1])
val_mask = np.zeros([n, ], dtype = 'bool')
for i in idx_val:
    y_val[i] = alabel[i]
    val_mask[i] = True

In [19]:
# Load data
#adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

/Users/Yiming/Desktop/Homework/IE 532/Project/gcn/utils.py:97: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [20]:
# Initialize weights
sess.run(tf.global_variables_initializer())

train_loss = []
validation_loss = []
train_accuracy = []
validation_accuracy = []


# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)
    
    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    validation_loss.append(cost)
    validation_accuracy.append(acc)
    train_loss.append(outs[1])
    train_accuracy.append(outs[2])

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and validation_loss[-1] > np.mean(validation_loss[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 246.09235 train_acc= 1.00000 val_loss= 0.04680 val_acc= 1.00000 time= 0.02910
Epoch: 0002 train_loss= 156.10789 train_acc= 1.00000 val_loss= 0.34567 val_acc= 1.00000 time= 0.00303
Epoch: 0003 train_loss= 93.22315 train_acc= 1.00000 val_loss= 0.35612 val_acc= 1.00000 time= 0.00308
Epoch: 0004 train_loss= 53.68621 train_acc= 1.00000 val_loss= 0.27515 val_acc= 1.00000 time= 0.00291
Epoch: 0005 train_loss= 37.78463 train_acc= 1.00000 val_loss= 0.03137 val_acc= 1.00000 time= 0.00304
Epoch: 0006 train_loss= 20.94411 train_acc= 1.00000 val_loss= 0.08031 val_acc= 1.00000 time= 0.00296
Epoch: 0007 train_loss= 8.98665 train_acc= 1.00000 val_loss= 0.10950 val_acc= 1.00000 time= 0.00284
Epoch: 0008 train_loss= 4.88086 train_acc= 1.00000 val_loss= 0.05397 val_acc= 1.00000 time= 0.00287
Epoch: 0009 train_loss= 2.51449 train_acc= 1.00000 val_loss= 0.00189 val_acc= 1.00000 time= 0.02035
Epoch: 0010 train_loss= 1.45273 train_acc= 1.00000 val_loss= 0.01212 val_acc= 1.00000 time= 

Epoch: 0109 train_loss= 0.00047 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00288
Epoch: 0110 train_loss= 0.00045 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00465
Epoch: 0111 train_loss= 0.00046 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00391
Epoch: 0112 train_loss= 0.00046 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00362
Epoch: 0113 train_loss= 0.00045 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00370
Epoch: 0114 train_loss= 0.00046 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.00455
Epoch: 0115 train_loss= 0.00045 train_acc= 1.00000 val_loss= 0.00028 val_acc= 1.00000 time= 0.01028
Epoch: 0116 train_loss= 0.00044 train_acc= 1.00000 val_loss= 0.00027 val_acc= 1.00000 time= 0.00385
Epoch: 0117 train_loss= 0.00044 train_acc= 1.00000 val_loss= 0.00027 val_acc= 1.00000 time= 0.00647
Epoch: 0118 train_loss= 0.00044 train_acc= 1.00000 val_loss= 0.00027 val_acc= 1.00000 time= 0.00469


Epoch: 0217 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00455
Epoch: 0218 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00900
Epoch: 0219 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00479
Epoch: 0220 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00370
Epoch: 0221 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00374
Epoch: 0222 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00614
Epoch: 0223 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00799
Epoch: 0224 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00704
Epoch: 0225 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00556
Epoch: 0226 train_loss= 0.00038 train_acc= 1.00000 val_loss= 0.00025 val_acc= 1.00000 time= 0.00562
